In [1]:
installPackage "NumericalAlgebraicGeometry";

stdio:2:1:(3): error: installPackage: the package is already installed and loaded from /Users/timcheek/Library/Application Support/Macaulay2/local/share/Macaulay2/


In [15]:
R2=CC[x];
R1=CC[x,t];
use R1;

In [16]:
taylorPoly=(f, x0, t0) -> {
    Fx=diff(x,f);   --yes, standard form is diff(x,f) for differentiate f wrt x
    Ft=diff(t,f);
    --yes, standard form is sub(f,{t=>2}), which works
    --also, need to do division in CC, otherwise get weird fraction field stuff
    g=x0-sub(sub(Ft,{x=>x0,t=>t0}),CC)/sub(sub(Fx,{x=>x0,t=>t0}),CC)*(t-t0);   
    return g;
};

In [17]:
runCorrector=(F, newT, guess)-> {
    f=sub(sub(F,{t=>newT}),R2);
    guess=point{{guess}};
    for i from 1 to 99 do (
        guess=newton(polySystem{f},guess);
        
        --print(guess.Coordinates);
    );
    return guess;
};

In [24]:
runAlgo=(F,x0,t0, newP1, newP2)->{
    approxFunc=taylorPoly(F,x0,t0);
    
    --now, substitute in other portal t1 value into g(t), to get approx zero for F(x,t1)=0 system
    --note that the complex number is represented by ii

    zeroGuess=sub(approxFunc,{t=>newP1});
    
    --now, apply newtons to the F(x,newP1)=0 system, using the zeroGuess estimate
    
    newX1= runCorrector(F, newP1, zeroGuess);
    
    --now, we have that (newX, newP) is a different solution to F(x,t)=0 (at least it should be)
    
    approxFunc=taylorPoly(F,newX1.Coordinates_0, newP1);
    zeroGuess=sub(approxFunc, {t=>newP2});
    newX2=runCorrector(F, newP2, zeroGuess);
    
    approxFunc=taylorPoly(F,newX2.Coordinates_0, t0);
    zeroGuess=sub(approxFunc, {t=>t0});
    return runCorrector(F, t0, zeroGuess);
    
};

In [49]:
F=(x^3-3*x-t)_R1;
x0=3;
t0=18;


guessPoint=runAlgo(F,x0,t0,random(CC)*10, random(CC)*-1*10);
guessPoint.Coordinates_0






o49 = - 1.5 - 1.93649*ii

o49 : R1
